In [ ]:
!pip install groq pydantic spacy[transformers] streamlit -qqq
!python -m spacy download en_core_web_trf -qqq
!pip install "numpy<2.0" -qqq

In [93]:
import os
from pydantic import BaseModel,Field, computed_field
from typing import List, Dict, Literal, Optional, ClassVar
import json
import spacy
import re
from groq import Groq
import pandas as pd
import streamlit as st
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("groq_api")

In [94]:
os.environ["GROQ_API_KEY"] = secret_value_0

In [95]:
nlp = spacy.load("en_core_web_trf")

In [96]:
transcript = '''Hello everyone, myself Muskan, studying in class 8th B section from Christ Public School.
I am 13 years old. I live with my family. There are 3 people in my family, me, my mother and my father.
One special thing about my family is that they are very kind hearted to everyone and soft spoken. One thing I really enjoy is play, playing cricket and taking wickets.
A fun fact about me is that I see in mirror and talk by myself. One thing people don't know about me is that I once stole a toy from one of my cousin.
My favorite subject is science because it is very interesting. Through science I can explore the whole world and make the discoveries and improve the lives of others.
Thank you for listening.'''
duration = 52

In [97]:
class SpeechMetrics:
    def __init__(self, text: str, seconds: int):
        self.doc = nlp(text)
        self.text = text
        self.seconds = seconds
        self.words = [token.text for token in self.doc if token.is_alpha]
        self.word_count = len(self.words)
    
    def get_speech_rate(self):
        wpm = round((self.word_count / self.seconds) * 60)
        if wpm > 161: category, score = "Too Fast", 2
        elif 141 <= wpm <= 160: category, score = "Fast", 6
        elif 111 <= wpm <= 140: category, score = "Ideal", 10
        elif 81 <= wpm <= 110: category, score = "Slow", 6
        else: category, score = "Too Slow", 2 # < 80
        
        return {"wpm": wpm, "category": category, "score": score}

    def get_clarity_score(self):
        fillers_list = ["um", "uh", "like", "you know", "so", "actually", "basically", 
                        "right", "i mean", "well", "kinda", "sort of", "okay", "hmm", "ah"]
        
        found_fillers = []
        for filler in fillers_list:
            pattern = r'\b' + re.escape(filler) + r'\b' 
            matches = re.findall(pattern, self.text, re.IGNORECASE)
            found_fillers.extend(matches)
            
        filler_count = len(found_fillers)
        filler_rate = round((filler_count / self.word_count) * 100) if self.word_count > 0 else 0
        
        if filler_rate <= 3: score = 15
        elif 4 <= filler_rate <= 6: score = 12
        elif 7 <= filler_rate <= 9: score = 9
        elif 10 <= filler_rate <= 12: score = 6
        else: score = 3
        
        return {"filler_words": found_fillers, "rate_percent": filler_rate, "score": score}

    def get_vocab_score(self):
        if self.word_count == 0: return 0
        distinct_words = set([w.lower() for w in self.words])
        ttr = len(distinct_words) / self.word_count
        
        if 0.9 <= ttr <= 1.0: return 10
        elif 0.7 <= ttr < 0.9: return 8
        elif 0.5 <= ttr < 0.7: return 6
        elif 0.3 <= ttr < 0.5: return 4
        else: return 2

In [98]:
metrics = SpeechMetrics(transcript, duration)
speech_data = metrics.get_speech_rate()
clarity_data = metrics.get_clarity_score()
vocab_score = metrics.get_vocab_score()

In [99]:
class Salutation(BaseModel):
    phrase_used: str = Field(..., description="The exact opening phrase used by the speaker (e.g., 'Hello everyone', 'Hi').")
    level: Literal["No Salutation", "Normal", "Good", "Excellent"] = Field(
        ..., 
        description=(
            "Assess the quality of the opening:\n"
            "- 'No Salutation': Starts immediately with content.\n"
            "- 'Normal': Simple 'Hi', 'Hello', 'Hey'.\n"
            "- 'Good': 'Good Morning', 'Good Afternoon', or inclusive 'Hello everyone'.\n"
            "- 'Excellent': Enthusiastic openers like 'Excited to be here', 'It's an honor to speak'."
        )
    )

    @computed_field
    def sal_score(self) -> int:
        mapping = {"No Salutation": 0, "Normal": 2, "Good": 4, "Excellent": 5}
        return mapping.get(self.level, 0)

class BasicDetails(BaseModel):
    name: Optional[str] = Field(None, description="The speaker's name. Look for 'Myself X', 'I am X'.")
    age: Optional[str] = Field(None, description="The speaker's age, extracted usually as a number near 'years old'.")
    school_class: Optional[List[str]] = Field(None, description="Educational details: Class/Grade and School Name. If adult, Job Title and Company.")
    family: Optional[str] = Field(None, description="Who constitutes the family structure (e.g., 'mother, father, me'). Do not describe personalities here.")
    hobbies: Optional[List[str]] = Field(None, description="Specific activities the speaker claims to enjoy or play (e.g., 'cricket', 'reading').")

    @computed_field
    def bd_score(self) -> int:
        found = [self.name, self.age, self.school_class, self.family, self.hobbies]
        return sum(1 for item in found if item) * 4

class ExtraDetails(BaseModel):
    about_family: Optional[str] = Field(None, description="Qualitative descriptions of the family (e.g., 'kind hearted', 'strict', 'supportive').")
    origin: Optional[str] = Field(None, description="Geographical origin or residence (City/State/Country).")
    ambition: Optional[str] = Field(None, description="Future goals, career aspirations, or broad desires (e.g., 'explore the world', 'become a doctor').")
    unique_fact: Optional[str] = Field(None, description="A specific fun fact, secret, or unique habit mentioned (e.g., 'talk to mirror', 'stole a toy').")
    strengths: Optional[List[str]] = Field(None, description="Self-identified strengths or favorite subjects that imply skill (e.g., 'Science is favorite', 'good at math').")

    @computed_field
    def ed_score(self) -> int:
        found = [self.about_family, self.origin, self.ambition, self.unique_fact, self.strengths]
        return sum(1 for item in found if item) * 2

class FlowSequence(BaseModel):
    is_order_followed: bool = Field(..., description="Boolean: True ONLY if the speech roughly follows: Greeting -> Bio/Intro -> Details -> Closing (Thank you).")
    
    @computed_field
    def flow_score(self) -> int:
        return 5 if self.is_order_followed else 0

In [100]:
class GrammarError(BaseModel):
    error_text: str = Field(..., description="The exact snippet from the text containing the error.")
    correction: str = Field(..., description="The grammatically correct version.")
    reason: str = Field(..., description="Linguistic explanation (e.g., 'Subject-Verb Agreement', 'Incorrect Pronoun Usage', 'Tense Mismatch').")

class GrammarAnalysis(BaseModel):
    errors: List[GrammarError] = Field(
        description="A list of OBJECTIVE grammatical errors."
    )
    
    def calculate_score(self, total_words: int) -> int:
        if total_words == 0: return 0
        raw_score = 1 - min((10 * len(self.errors)) / total_words, 1)
        if raw_score >= 0.9: return 10
        elif 0.7 <= raw_score < 0.9: return 8
        elif 0.5 <= raw_score < 0.7: return 6
        elif 0.3 <= raw_score < 0.5: return 4
        return 2

In [101]:
class Engagement(BaseModel):
    sentiment_label: Literal["Positive", "Neutral", "Negative"] = Field(..., description="Overall tone: 'Positive' (Enthusiastic, Hopeful), 'Neutral' (Factual, Robotic), 'Negative' (Sad, Complaining).")
    positivity_probability: float = Field(..., description="A float between 0.0 and 1.0 representing the confidence that the tone is positive/engaging.")

    @computed_field
    def eng_score(self) -> int:
        val = self.positivity_probability
        if val >= 0.9: return 15
        elif 0.7 <= val < 0.9: return 12
        elif 0.5 <= val < 0.7: return 9
        elif 0.3 <= val < 0.5: return 6
        return 3

In [102]:
class EvaluationResult(BaseModel):
    salutation: Salutation
    basic_details: BasicDetails
    extra_details: ExtraDetails
    flow: FlowSequence
    grammar: GrammarAnalysis
    engagement: Engagement

In [103]:
SYSTEM_PROMPT = """
You are an expert Linguistic Evaluator and Speech Coach AI. 
Your task is to analyze the provided speech transcript and extract structured data regarding the speaker's content, grammar, and engagement.

You must output a JSON object that strictly adheres to the schema provided below.

### INSTRUCTIONS FOR EXTRACTION:

1. **Salutation**: Identify how the speaker opens. Rank the quality based on warmth and professionalism.
2. **Basic Details**: Extract factual datas such as name, age, school, class, hobbies. 
3. **Extra Details**: Look for qualitative descriptors (adjectives about family), origin, ambitions, unique facts and strengths.
4. **Flow**: Check if the speech has a logical beginning (Greeting), middle (Content), and end (Closing/Thanks).
5. **Grammar**: Identify only major grammatical errors that affect clarity or correctness. Ignore minor conversational mistakes.
6. **Engagement**: Analyze the sentiment. Is the speaker sharing personal details enthusiastically?

### OUTPUT SCHEMA:
{schema}
""".format(schema=json.dumps(EvaluationResult.model_json_schema(), indent=2))

In [104]:
client = Groq()

In [105]:
try:
    completion = client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": transcript}
        ],
        temperature=0.0,
        response_format={"type": "json_object"}
    )

    raw_json = completion.choices[0].message.content
    data = EvaluationResult.model_validate_json(raw_json)

    grammar_score = data.grammar.calculate_score(metrics.word_count)
    content_score = (data.salutation.sal_score + data.basic_details.bd_score + 
                     data.extra_details.ed_score + data.flow.flow_score)

    final_report = {
        "Content & Structure": {
            "details": {
                "Salutation Level": data.salutation.model_dump(),
                "basic": data.basic_details.model_dump(),
                "extra": data.extra_details.model_dump(),
                "flow": data.flow.model_dump()
            },
            "score": content_score
        },
        "Speech Rate": speech_data,
        "Language & Grammar": {
            "vocabulary_score": vocab_score,
            "grammar_errors_found": [e.model_dump() for e in data.grammar.errors],
            "grammar_score": grammar_score,
            "total_section_score": vocab_score + grammar_score
        },
        "Clarity": clarity_data,
        "Engagement": {
            "sentiment": data.engagement.sentiment_label,
            "score": data.engagement.eng_score
        }
    }

    print(json.dumps(final_report, indent=2))

except Exception as e:
    print(f"Error interacting with Groq or parsing data: {e}")

{
  "Content & Structure": {
    "details": {
      "Salutation Level": {
        "phrase_used": "Hello everyone",
        "level": "Good",
        "sal_score": 4
      },
      "basic": {
        "name": "Muskan",
        "age": "13",
        "school_class": [
          "8th B",
          "Christ Public School"
        ],
        "family": "mother, father, me",
        "hobbies": [
          "cricket",
          "taking wickets"
        ],
        "bd_score": 20
      },
      "extra": {
        "about_family": "they are very kind hearted to everyone and soft spoken",
        "origin": null,
        "ambition": "explore the whole world and make discoveries to improve the lives of others",
        "unique_fact": "I look in the mirror and talk to myself",
        "strengths": [
          "science"
        ],
        "ed_score": 8
      },
      "flow": {
        "is_order_followed": true,
        "flow_score": 5
      }
    },
    "score": 37
  },
  "Speech Rate": {
    "wpm": 150,
    

In [106]:
sal_score = data.salutation.model_dump()['sal_score']
bd_score = data.basic_details.model_dump()['bd_score']
ed_score = data.extra_details.model_dump()['ed_score']
kw_presence = bd_score + ed_score
flow = data.flow.model_dump()['flow_score']
weight_cs = sal_score + kw_presence + flow   
srate = speech_data['score']
lang_score = vocab_score
gra_score = grammar_score
wweight_lg = lang_score + gra_score          
cla = clarity_data['score']
eng = data.engagement.eng_score

In [107]:
df = pd.DataFrame([
    ["Content & Structure", "Salutation Level", sal_score, weight_cs],
    ["Content & Structure", "Key word Presence", kw_presence, weight_cs],
    ["Content & Structure", "Flow", flow, weight_cs],
    ["Speech Rate", "Speech rate (words/minute)", srate, srate],
    ["Language & Grammar", "Grammar errors count", gra_score, wweight_lg],
    ["Language & Grammar", "Vocabulary richness", lang_score, wweight_lg],
    ["Clarity", "Filler Word Rate", cla, cla],
    ["Engagement", "Sentiment/positivity", eng, eng],
], columns=["Criteria", "Metric", "Score Obtained", "Weightage"])

In [108]:
def pretty_print(df):
    last_criteria = None

    print("\nCriteria".ljust(22), "Metric".ljust(60), "Score".ljust(10), "Weightage")
    print("-" * 110)
    
    section_scores = {}

    for index, row in df.iterrows():
        criteria = row["Criteria"]
        metric = row["Metric"]
        score = row["Score Obtained"]
        first_row = df[df["Criteria"] == criteria].index[0]
        weightage = row["Weightage"] if index == first_row else ""
        if last_criteria is not None and criteria != last_criteria:
            print("-" * 110)
        criteria_display = criteria.ljust(22) if index == first_row else " " * 22
        print(criteria_display, metric.ljust(60), str(score).ljust(10), str(weightage))
        section_scores.setdefault(criteria, 0)
        section_scores[criteria] += score
        last_criteria = criteria

    print("-" * 110)

    total_score = sum(section_scores.values())       
    final_percentage = round(total_score, 2)         

    print(f"{'Final Score (out of 100):'.ljust(94)} {final_percentage}")
    print("-" * 110)

In [109]:
pretty_print(df)


Criteria              Metric                                                       Score      Weightage
--------------------------------------------------------------------------------------------------------------
Content & Structure    Salutation Level                                             4          37
                       Key word Presence                                            28         
                       Flow                                                         5          
--------------------------------------------------------------------------------------------------------------
Speech Rate            Speech rate (words/minute)                                   6          6
--------------------------------------------------------------------------------------------------------------
Language & Grammar     Grammar errors count                                         6          12
                       Vocabulary richness                                   